In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

# Nueva sección

In [ ]:
train_data = pd.read_csv(train_file_path, sep="\t", header=None, names=["label", "message"])
test_data = pd.read_csv(test_file_path, sep="\t", header=None, names=["label", "message"])

train_data["label"] = train_data["label"].map({"ham": 0, "spam": 1})
test_data["label"] = test_data["label"].map({"ham": 0, "spam": 1})

vocab = set()
for message in train_data["message"]:
    for word in message.split():
        vocab.add(word.lower())
vocab = list(vocab)
word_index = {word: idx for idx, word in enumerate(vocab)}

def text_to_vector(text):
    vector = np.zeros(len(vocab))
    for word in text.split():
        if word.lower() in word_index:
            vector[word_index[word.lower()]] += 1
    return vector

X_train = np.array([text_to_vector(msg) for msg in train_data["message"]])
y_train = train_data["label"].values
X_test = np.array([text_to_vector(msg) for msg in test_data["message"]])
y_test = test_data["label"].values

model = keras.Sequential([
    keras.layers.Dense(64, activation="relu", input_shape=(len(vocab),)),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation="sigmoid")
])

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  vector = text_to_vector(pred_text)
  prediction = model.predict(np.array([vector]))[0][0]
  label = "spam" if prediction > 0.5 else "ham"
  prob = prediction if label == "spam" else 1 - prediction
  return [float(prob), label]

# Probar
pred_text = "how are you doing today?"
print(predict_message(pred_text))

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
